In [22]:
import numpy as np
import scipy.sparse as sp

from src.utils import RecommenderModel, open_data, make_submission

## Opening the data

In [23]:
urm, icm = open_data()
urm, icm

(<Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 1764607 stored elements and shape (35736, 38121)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 2940040 stored elements and shape (38121, 94331)>)

## Building the Top Recommender model
Seen items are removed from the recommendations

In [24]:
class TopPopRecommender(RecommenderModel):
	def __init__(self):
		super(TopPopRecommender, self).__init__()
		self.urm_train: sp.csr_matrix | None = None
		self.items_popularity_ranking: np.ndarray | None = None

	def fit(self, urm: sp.csr_matrix) -> None:
		self.urm_train = urm
		items_popularity = np.ediff1d(urm.tocsc().indptr)
		self.items_popularity_ranking = np.argsort(-items_popularity)

	def recommend(self, user_id: int, at: int = 10) -> np.ndarray:
		seen_items = self.urm_train.indices[self.urm_train.indptr[user_id]:self.urm_train.indptr[user_id + 1]]
		unseen_items_mask = np.isin(self.items_popularity_ranking, seen_items, assume_unique=True, invert=True)

		return self.items_popularity_ranking[unseen_items_mask][:at]

In [25]:
top_pop = TopPopRecommender()
top_pop.fit(urm)

## Testing the model

In [26]:
make_submission(top_pop, "top_pop_recommendation.csv")

KeyboardInterrupt: 